# Chatbot 

Web-based interface of the chatbot interface: https://gonzalorecio.com/chatbot/robot.html

In [4]:
import requests

In [15]:
class ChatbotFace:
    chatbot_mood_API   = 'https://chatbot-mood.herokuapp.com/mood'
    chatbot_status_API = 'https://chatbot-mood.herokuapp.com/internal_state'

    def change_mood(self, mood):
        req_obj = {'action': mood}
        x = requests.post(self.chatbot_mood_API, data = req_obj)
        # print(x)
        
    def change_status(self, status):
        req_obj = {'status': status}
        x = requests.post(self.chatbot_status_API, data = req_obj)
        
    def neutral(self):
        self.change_mood('neutral')
        
    def happy(self):
        self.change_mood('happy')
        
    def sad(self):
        self.change_mood('sad')
        
    def angry(self):
        self.change_mood('angry')
        
    def focused(self):
        self.change_mood('focused')
        
    def confused(self):
        self.change_mood('confused')
    
    def start_blinking(self):
        self.change_mood('start_blinking')
        
    def stop_blinking(self):
        self.change_mood('stopt_blinking')
        
    def status_listening(self):
        self.change_status('listening')
        
    def status_thinking(self):
        self.change_status('thinking')
        
    def status_none(self):
        self.change_status('')

In [16]:
chatbot = ChatbotFace()

In [21]:
chatbot.confused()

In [20]:
chatbot.happy()

In [26]:
chatbot.status_thinking()

In [34]:
chatbot.status_listening()

In [35]:
chatbot.status_none()

In [36]:
chatbot.neutral()

# Sentiment

In [14]:
from sentiments.BERTGRU_model import BERTGRUSentiment
import requests
import torch
from transformers import BertModel, BertTokenizer

In [15]:
import sys
print(sys.version)

3.7.4 (v3.7.4:e09359112e, Jul  8 2019, 14:54:52) 
[Clang 6.0 (clang-600.0.57)]


In [17]:
class ChatbotFace:
    '''Chatbot face according to sentiment analysis. '''
    chatbot_mood_API   = 'https://chatbot-mood.herokuapp.com/mood'
    chatbot_status_API = 'https://chatbot-mood.herokuapp.com/internal_state'

    def change_mood(self, mood):
        req_obj = {'action': mood}
        x = requests.post(self.chatbot_mood_API, data = req_obj)
        # print(x)
        
    def change_status(self, status):
        req_obj = {'status': status}
        x = requests.post(self.chatbot_status_API, data = req_obj)
    
    def status_custom(self,text):
      self.change_status(text)
    
    def predict_sentiment(self,sentence):
        model_path = '/content/drive/MyDrive/CIR/BERT_sentiment.pt'
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        bert = BertModel.from_pretrained('bert-base-uncased')
        model = BERTGRUSentiment(bert,
                                  hidden_dim=256,
                                  output_dim=1,
                                  n_layers=2,
                                  bidirectional=True,
                                  dropout=0.25)
        model.to(device)
        model.load_state_dict(torch.load(model_path))
        odel.eval()
        tokens = tokenizer.tokenize(sentence)
        tokens = tokens[:max_input_length-2]
        indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
        tensor = torch.LongTensor(indexed).to(device)
        tensor = tensor.unsqueeze(0)
        prediction = torch.sigmoid(model(tensor))
    
        return prediction.item()
    
    def mood_prediction(self,utterance):
      self.change_mood('start_blinking')
      score = self.predict_sentiment(utterance)
      print(score)
      self.status_custom(utterance)
      if '?' not in utterance:
        if score <= 0.55 and score> 0.45: 
          self.change_mood('neutral')
        elif score <= 1 and score> 0.55:
          self.change_mood('happy')
        elif score <= 0.45 and score> 0:
          self.change_mood('sad')
      else: 
        self.change_mood('confused')
      
    def start_blinking(self):
        self.change_mood('start_blinking')
        
    def stop_blinking(self):
        self.change_mood('stopt_blinking')
        
    def status_listening(self):
        self.change_status('listening')
        
    def status_thinking(self):
        self.change_status('thinking')
        
    def status_none(self):
        self.change_status('')

In [18]:
chatbot = ChatbotFace()
chatbot.mood_prediction('This film is great')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/CIR/BERT_sentiment.pt'


## Speech

In [30]:
from speech import google_speech as speech

In [13]:
# Test
speech.speech_to_audio(lang="en-US")

Listening...
Recognizing...
hola hola


'hola hola'

In [10]:
# Test
s = 'what are you saying?? :D'
speech.text_to_speech(s, speed=150, voice=1)

## Translation

In [31]:
from google_trans_new import google_translator  
translator = google_translator()  
translate_text = translator.translate('สวัสดีจีน', lang_tgt='es')  
print(translate_text)
#output: Hello china

Hola china 


# Chatbot

In [33]:
import tensorflow
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from google_trans_new import google_translator  

class Chatbot:
    def __init__(self, lang='en'):
        print('Loading model and tokenizers...')
        self.model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
        self.face = ChatbotFace()
        
        
        
        self.reset_chatbot(lang=lang)
        print('Chatbot ready.')
    
    def reset_chatbot(self, lang='en'):
        self.chat_history_ids = self.tokenizer.encode(self.tokenizer.bos_token, return_tensors='pt')
        if lang=='es':
            self.voice = 0 # 0: spanish female, 1: english female, 2: english male
        else:
            self.voice = 1 # 0: spanish female, 1: english female, 2: english male
        self.translator = google_translator()  
        self.lang = lang
        
    def listen_and_get_question(self):
        face.status_listening()
        lang = 'en-US' if self.lang=='en' else 'es-ES'
        question = speech.speech_to_audio()
        if self.lang=='es':
            question = self.translate(question, lang='en')
        face.status_none()
        return question
    
    def decode(self, token_ids):
        return self.tokenizer.decode(token_ids, skip_special_tokens=True)
    
    def generate_answer(self, question):
        face.status_thinking()
        question_ids = self.tokenizer.encode(question + self.tokenizer.eos_token, return_tensors='pt')
        input_ids = torch.cat([self.chat_history_ids, question_ids], dim=-1)
        self.chat_history_ids = self.model.generate(input_ids, max_length=1000, pad_token_id=self.tokenizer.eos_token_id)
        answer_ids = self.chat_history_ids[:, input_ids.shape[-1]:][0]
        text = self.decode(answer_ids)
        if self.lang == 'es':
            text = self.translate(text, lang='es')
        face.status_none()
        return text
    
    def get_sentiment_analysis(self, text):
        return True
        
    def speak(self, text):
        self.face.happy()
        speech.text_to_speech(text, speed=150, voice=self.voice)
        self.face.neutral()
        
    def translate(self, text, lang):
        translated_text = self.translator.translate(text, lang_tgt=lang)
        if type(translated_text) == list:
            return translated_text[0]
        return translated_text
    
    def no_understand(self):
        text = "Sorry, I couldn't understand you"
        if self.lang == 'es':
            text = self.translete(text, lang='es')
        self.speak(text)
    
    def run_chat(self):
        self.reset_chatbot(lang=self.lang)
        self.face.neutral()
        question = ''
        while(question != 'goodbye'):
            question = self.listen_and_get_question()
            if question is None:
                print('Fallo')
                self.no_understand()
                continue
            print('Question:', question)
            answer = self.generate_answer(question)
            print('Answer:', answer)
            self.speak(answer)
            
#             self.reset_chatbot(lang=self.lang)
            print(self.chat_history_ids)
        

In [20]:
chatbot = Chatbot(lang='es')
# chatbot.run()

Loading model and tokenizers...
Chatbot ready.


In [21]:
chatbot.reset_chatbot(lang='en')

In [23]:
chatbot.run_chat()

Listening...
Recognizing...
hello
Question: hello
Answer: Hello!
tensor([[50256, 31373, 50256, 15496,  5145, 50256]])
Listening...
Recognizing...
how are you
Question: how are you
Answer: I'm good, how are you?
tensor([[50256, 31373, 50256, 15496,  5145, 50256,  4919,   389,   345, 50256,
            40,  1101,   922,   837,   703,   389,   345,  5633, 50256]])
Listening...


KeyboardInterrupt: 